# Hyperparameter Tuning using HyperDrive


In [13]:
from azureml.core import Workspace, Environment
from azureml.core import Experiment
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core.conda_dependencies import CondaDependencies

from azureml.train.hyperdrive import BayesianParameterSampling, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import normal, uniform, choice

import shutil
import os

## Dataset

In [3]:
data_path = "https://raw.githubusercontent.com/jaircastruita/nd00333-capstone/master/starter_file/data/WA_Fn-UseC_-HR-Employee-Attrition.csv"

ibm_ds = Dataset.Tabular.from_delimited_files(path=data_path)

In [4]:
ws = Workspace.from_config()
experiment_name = 'dnn-experiment'

experiment=Experiment(ws, experiment_name)

## Compute Target

In [5]:
cluster_name = "hd-dnn-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Reusing already created compute target")

except ComputeTargetException:
    print("Creating compute target...")
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6", max_nodes=6)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Creating compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-15T22:36:27.698000+00:00', 'errors': None, 'creationTime': '2021-02-15T22:36:25.305400+00:00', 'modifiedTime': '2021-02-15T22:36:41.159113+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 6, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [6]:
project_folder = "./dnn-attrition"
os.makedirs(project_folder, exist_ok=True)
shutil.copy("dnn-train.py", project_folder)

'./dnn-attrition/dnn-train.py'

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [18]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-defaults
  - torch==1.6.0
  - scikit-learn

Writing conda_dependencies.yml


In [19]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# early_termination_policy = <your policy here>

param_sampling = BayesianParameterSampling({
    "learning_rate": uniform(0.001, 0.1),
    "num_epochs": choice(range(100, 5000)),
    "layer1": choice(range(10, 100)),
    "layer2": choice(range(10, 100)),
})

conda_env = Environment.from_conda_specification(name="PyTorch-1.6-GPU-scikit-learn", file_path="conda_dependencies.yml")
# Specify a GPU base image
# pytorch_env.docker.enabled = True
# pytorch_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

src = ScriptRunConfig(source_directory=project_folder,
                      script="dnn-train.py",
                      arguments=["--num_epochs", 100, "--layer1", 30, "--layer2", 15, "--learning_rate", 0.001,"--output_dir", "./outputs"],
                      compute_target=compute_target,
                      environment=conda_env)

hyperdrive_run_config = HyperDriveConfig(run_config=src, 
                                         hyperparameter_sampling=param_sampling, 
                                         primary_metric_name='best_test_acc', 
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=8,
                                         max_concurrent_runs=4)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:80.


In [20]:
run = experiment.submit(src)
print(run)

Run(Experiment: dnn-experiment,
Id: dnn-experiment_1613430933_79335d39,
Type: azureml.scriptrun,
Status: Starting)


In [21]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: dnn-experiment_1613430933_79335d39
Web View: https://ml.azure.com/experiments/dnn-experiment/runs/dnn-experiment_1613430933_79335d39?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-138979/workspaces/quick-starts-ws-138979

Streaming azureml-logs/20_image_build_log.txt

2021/02/15 23:15:41 Downloading source code...
2021/02/15 23:15:42 Finished downloading source code
2021/02/15 23:15:43 Creating Docker network: acb_default_network, driver: 'bridge'
2021/02/15 23:15:43 Successfully set up Docker network: acb_default_network
2021/02/15 23:15:43 Setting up Docker configuration...
2021/02/15 23:15:44 Successfully set up Docker configuration
2021/02/15 23:15:44 Logging in to registry: 7cd3b6cefbf14c83a9f318fe92daa130.azurecr.io
2021/02/15 23:15:45 Successfully logged into 7cd3b6cefbf14c83a9f318fe92daa130.azurecr.io
2021/02/15 23:15:45 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/0

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.\n\tReason: Job failed with non-zero exit Code",
        "messageFormat": "{Message}",
        "messageParameters": {
            "Message": "AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.\n\tReason: Job failed with non-zero exit Code"
        },
        "details": [],
        "innerError": {
            "code": "BadArgument",
            "innerError": {
                "code": "AmlComputeBadRequest"
            }
        }
    },
    "correlation": {
        "operation": null,
        "request": "1c59042371b470a5"
    },
    "environment": "southcentralus",
    "location": "southcentralus",
    "time": "2021-02-15T23:26:24.048694Z",
    "comp

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status == "Completed")

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)


In [ ]:

print('Best Run is:\n  Validation accuracy: {0:.5f} \n  Learning rate: {1:.5f} \n Layer 1: {} \n layer 2: {}'.format(
        best_run_metrics['best_test_acc'][-1],
        best_run_metrics['learning_rate'],
        best_run_metrics['layer1'],
        best_run_metrics['layer2'])
     )

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='pytorch-attrition', 
                                model_path='outputs/dnn-model.pt')
print(model.name, model.id, model.version, sep='\t')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service